In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
def previsao_consumo_diario(data_inicio, 
                            data_fim, 
                            dias_rampa, 
                            valor_arr, 
                            cliente, 
                            oportunidade = '',
                            tipo_forecast = '',
                            account_manager='',
                            pct_inicial_rampa=0.01, 
                            pct_final_rampa=0.90):
    idx = pd.date_range(data_inicio, data_fim)
    ramp_linspace = np.linspace(start=pct_inicial_rampa, stop=pct_final_rampa, num=dias_rampa)
    
    if len(idx) - dias_rampa >= 0:
        ramp_array = np.append(ramp_linspace, np.ones(len(idx) - dias_rampa))
    else:
        ramp_array = ramp_linspace[0: len(idx)]
    
    valor_diario = valor_arr / 365
    previsao_consumo_array = valor_diario * ramp_array
    
    df = pd.DataFrame(
        {
            'Consumo Projetado USD': previsao_consumo_array,
            'Cliente': cliente,
            'Oportunidade': oportunidade,
            'Tipo Forecast': tipo_forecast,
            'Account Manager': account_manager
        },
        index=idx
    )
    df.index.name = 'Data'
    
    return df

In [3]:
excel_files = [file for file in glob.glob('./lista_oportunidades/*.xlsx')]
excel_df_list = [pd.read_excel(excel_file) for excel_file in excel_files]
lista_oportunidades_unificada = pd.concat(excel_df_list)
lista_oportunidades_unificada.to_excel('./Lista Unificada de Oportunidades Territorio PS Federal.xlsx', index=False)
oportunidades = lista_oportunidades_unificada.to_dict(orient='records')

In [4]:
# excel_files = [file for file in glob.glob('./lista_oportunidades/*.xlsx')]
# dict_list = [pd.read_excel(excel_file).to_dict(orient='records')
#                 for excel_file in excel_files]
# Flatten lista de listas 
# oportunidades = list(itertools.chain(*dict_list))

In [5]:
lista_previsoes = [previsao_consumo_diario(**op) for op in oportunidades]

In [6]:
previsoes_df = pd.concat(lista_previsoes)

In [7]:
previsoes_df

,Consumo Projetado USD,Cliente,Oportunidade,Tipo Forecast,Account Manager
Data,,,,,
2021-06-01,14.865753,Min. Da Saúde,MS OCI Bootcamp,Won,Sarah Paes
2021-06-02,14.865753,Min. Da Saúde,MS OCI Bootcamp,Won,Sarah Paes
2021-06-03,14.865753,Min. Da Saúde,MS OCI Bootcamp,Won,Sarah Paes
2021-06-04,14.865753,Min. Da Saúde,MS OCI Bootcamp,Won,Sarah Paes
2021-06-05,14.865753,Min. Da Saúde,MS OCI Bootcamp,Won,Sarah Paes
...,...,...,...,...,...
2022-05-27,168.657534,DATAPREV,Weblogic Dev Cloud,Pipeline,Alexandre Viana
2022-05-28,171.616438,DATAPREV,Weblogic Dev Cloud,Pipeline,Alexandre Viana
2022-05-29,174.575342,DATAPREV,Weblogic Dev Cloud,Pipeline,Alexandre Viana


In [8]:
with pd.ExcelWriter('Previsão de Consumo Accounts PS Federal.xlsx') as writer:
    previsoes_df.to_excel(writer, sheet_name='Previsão de Consumo')